# Sonification of Chromagrams
In this notebook, we illustrate the sonification of chromagrams using the ```libsoni.core.chroma``` module. A chromagram can be seen as a representation of the distribution of musical pitch classes, a feature often used in MIR Applications.

TODO: One Octave Shepard tone example

In [ ]:
import os
import sys
import numpy as np
import librosa, librosa.display
import pandas as pd

from matplotlib import pyplot as plt
import IPython.display as ipd

from libsoni.util.utils import mix_sonification_and_original
from libsoni.core.chroma import sonify_chromagram
from libsoni.core.methods import generate_shepard_tone

sys.path.append('..')
import libfmp.b
import libfmp.c8

%matplotlib inline

Fs = 22050
H = 1024

## Simple Scenario: C Major Cadence
To start with a simple example, let's look at the chords of a C major cadence: **C:maj**, **F:maj** and **G:maj**.
<img src="figures/demo_chroma/C_Major_Cadence.png" alt="C-Major-Triad" width="500">

In [ ]:
C_Maj_Cadence_Chords_pitchclasses = [(0,4,7), (5,9,0), (2,7,11), (0,4,7)]

starts = [0, 1, 2, 3]

durations = [0.75, 0.75, 0.75, 0.75]

Fs_frame = Fs / H



Chromagram_C_Maj_Cadence = np.zeros((12,int((starts[-1]+durations[-1])*Fs/H)))


for i in range(len(starts)):
    Chromagram_C_Maj_Cadence[C_Maj_Cadence_Chords_pitchclasses[i],int(starts[i]*Fs/H):int((starts[i]+durations[i])*Fs/H)]=1

title = 'Chromagram for C Major Cadence'
libfmp.b.plot_chromagram(Chromagram_C_Maj_Cadence, Fs=Fs_frame, title=title, figsize=(8, 2.5))
plt.show()

### Sonification using Shepard Tones

In [ ]:
shepard_tone = generate_shepard_tone()

ipd.display(ipd.Audio(shepard_tone, rate=Fs))

### Sonified C Major Cadence

In [ ]:
# Sonification using libsoni
x_chroma_ann = sonify_chromagram(Chromagram_C_Maj_Cadence,H=512,fs=Fs)

print('Sonification of chroma-based respresentation of the C Major Cadence:')
ipd.display(ipd.Audio(x_chroma_ann, rate=Fs))

### Customizing the Sonification

Some sound changing parameters can be passed to ```sonify_chromagram``` in order to change the shepard tones. 

- ```pitch_range```: Tuple defining which pitch range to encounter in shepard tone
- ```filter```: bool, deciding, if shepard tones are filtered or not
- ```f_center``` : float, center_frequency in Hertz for bell-shaped filter
- ```octave_cutoff```: int, determines, at which multiple of f_center, the harmonics get attenuated by 2.
Let's show some examples:

In [ ]:
# Shepard tone, default
shepard_tone = generate_shepard_tone()
print('Shepard tone, default:')
ipd.display(ipd.Audio(shepard_tone, rate=Fs))

# Shepard tone with small, high pitch range
shepard_tone_small_pitch_range_high = generate_shepard_tone(pitch_range=(60,84))
print('Shepard tone with small, high pitch range:')
ipd.display(ipd.Audio(shepard_tone_small_pitch_range_high, rate=Fs))

# Shepard tone with small, low pitch range
shepard_tone_small_pitch_range_low = generate_shepard_tone(pitch_range=(36,60))
print('Shepard tone with small, low pitch range:')
ipd.display(ipd.Audio(shepard_tone_small_pitch_range_low, rate=Fs))

# Narrow filtered Shepard tone with low center frequency:
shepard_tone_filtered_low = generate_shepard_tone(filter=True, f_center=100, octave_cutoff=1)
print('Narrow filtered Shepard tone with low center frequency:')
ipd.display(ipd.Audio(shepard_tone_filtered_low, rate=Fs))

# Wide filtered Shepard tone with high center frequency:
shepard_tone_filtered_low = generate_shepard_tone(pitch_range=(50,127), filter=True, f_center=1000, octave_cutoff=3)
print('Wide filtered Shepard tone with high center frequency:')
ipd.display(ipd.Audio(shepard_tone_filtered_low, rate=Fs))

## Scenario: *Frühlingstraum (Dream of Spring)* by *Franz Schubert*
<img src="figures/demo_chroma/FMP_B_Sonify_Pitch_Schubert_D911-11_A.png" alt="Frühlingstraum" width="600">

In [ ]:
schubert_audio, Fs = librosa.load(os.path.join('data_audio', 'demo_chroma', 'FMP_B_Sonify_Pitch_Schubert_D911-11_SC06.wav'), sr=Fs)

print('Frühlingstraum by Franz Schubert:')
ipd.display(ipd.Audio(schubert_audio,rate=Fs))

### Preparing the data

In [ ]:
schubert_df = pd.read_csv(os.path.join('data_csv','demo_chroma','FMP_B_Sonify_Pitch_Schubert_D911-11_SC06.csv'), sep=';')

note_list = [[start, duration, pitch, vel, ins] for i, 
             (start, duration, pitch, vel, ins) in schubert_df.iterrows()]

num_frames = int(len(schubert_audio) / H)
Fs_frame = Fs / H
C_annotated_schubert = libfmp.b.list_to_chromagram(note_list, num_frames, Fs_frame)
ipd.display(schubert_df[0:5])

In [ ]:
libfmp.b.plot_chromagram(C_annotated_schubert, Fs=Fs_frame, title='Frühlingstraum, Franz Schubert', figsize=(8, 2.5))
plt.show()

# Sonified Chromagram from annotation
c_annotated_schubert_sonified = sonify_chromagram(C_annotated_schubert,H=H,fs=Fs)
print('Sonification of annotated Chromagram:')
ipd.display(ipd.Audio(c_annotated_schubert_sonified, rate=Fs))

schubert_annotated_sonified_w_original = mix_sonification_and_original(sonification=c_annotated_schubert_sonified,
                                                         original_audio=schubert_audio,
                                                         gain_lin_sonification=1,
                                                         gain_lin_original_audio=1)

print('Sonification of original Audio and annotated Chromagram:')
ipd.display(ipd.Audio(schubert_annotated_sonified_w_original, rate=Fs))

In [ ]:
# Computation of Chromagram from audio using librosa
N = 2048
H = 1024
C_schubert = librosa.feature.chroma_stft(y=schubert_audio, sr=Fs, tuning=0, norm=2, hop_length=H, n_fft=N)

libfmp.b.plot_chromagram(C_schubert, Fs=Fs_frame, title='Frühlingstraum, Franz Schubert', figsize=(8, 2.5))
plt.show()


c_schubert_sonified = sonify_chromagram(C_schubert,H=H,fs=Fs)


print('Sonification of computed Chromagram:')
ipd.display(ipd.Audio(c_schubert_sonified, rate=Fs))


schubert_sonified_w_original = mix_sonification_and_original(sonification=c_schubert_sonified,
                                                             original_audio=schubert_audio,
                                                             gain_lin_sonification=1,
                                                             gain_lin_original_audio=1)

print('Sonification of computed Chromagram and annotated Chromagram:')
ipd.display(ipd.Audio(schubert_sonified_w_original, rate=Fs))